# Prepare environment

## Configure jupyter

* Install libs

In [ ]:
%%capture
!pip install cmake
!pip install deepface

In [ ]:
!pip install dlib

In [ ]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt

# run all code cells of the specified notebook
%run ../face/libs/image_utils.ipynb

# Use DeepFace library

In [ ]:
from deepface import DeepFace

backend_names = ["opencv", "ssd", "dlib", "mtcnn", "retinaface", "mediapipe"]
model_names = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "ArcFace"]
# model_names = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "ArcFace", "DeepID", "Dlib", "SFace"]

target_size=(224, 224)

face_path = '/data/faces/'
person_path = face_path + 'elon_musk/'
face1 = person_path + 'image1.jpeg'
face2 = person_path + 'image2.jpeg'
face_double = person_path + 'image_double.jpeg'

In [ ]:
# READ GRAYSCALE
img = img_read_cv(face1, cv2.IMREAD_GRAYSCALE)
img = img_convert_color(img)
img_show_using_color(img, 'gray')

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(15, 10))
axs = axs.flatten()

for index, backend_name in enumerate(backend_names):
    try:
        face = DeepFace.detectFace(face_double, target_size=target_size, detector_backend=backend_name)
        img_show_axis(img=face, axis=axs[index], title=backend_name)
    except:
        pass
plt.show()

In [ ]:
for model_name in model_names:
    result = DeepFace.verify(
        img1_path = face1,
        img2_path = face2,
        model_name=model_name
    )
    print(model_name, '=>', result)

    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle(f'Verified {result["verified"]} - Distance {result["distance"]:0.4}')

    img_show_axis(img=face1, axis=axs[0], func=plt.imread)
    img_show_axis(img=face2, axis=axs[1], func=plt.imread)
    plt.show()

In [ ]:
result = DeepFace.find(img_path=face1, db_path=face_path)
result

In [ ]:
from glob import glob
imgs = glob(person_path + "*")

for img_path in imgs:
    analysis = DeepFace.analyze(
        img_path=img_path, 
        detector_backend=backend_names[4]
    )
    emo_df = pd.DataFrame(analysis[0]["emotion"], index=[0]).T.rename(
        columns={0: "prediction"}
    )
    img_show_axis_values(img_path, emo_df.sort_values("prediction"))
    plt.show()